In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql import SQLContext

import wget
import os
from datetime import datetime

In [2]:
!mkdir /home/ebbcf/datasets/

In [3]:
!mkdir /home/ebbcf/logs/

In [4]:
filepath = '/home/ebbcf/datasets/' 

In [5]:
logpath = '/home/ebbcf/logs/' 

In [6]:
json_part_files = [
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00000.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00001.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00002.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00003.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00004.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00005.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00006.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00007.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00008.json.gz',
'https://d3l36jjwr70u5l.cloudfront.net/data-engineer-test/part-00009.json.gz']

In [7]:
etapa1 = {}

In [8]:
for json in json_part_files:  
    wget.download(json, out=filepath + json.split("/")[4])

In [9]:
def unique_sessions(filepath, file):
    df = spark.read.json('file://' + filepath +  file)
    return df.select('anonymous_id').distinct().count()

In [10]:
for file in os.listdir(filepath):
    if file[0:4] == 'part':
        etapa1[file] = unique_sessions(filepath, file)  
        f = open(logpath + 'log' + datetime.now().strftime("%d_%m_%Y") + '.txt', "a")
        f.write("Finalmente tem log: " + str(etapa1) + '\n')
        f.close()

In [11]:
etapa1

{'part-00000.json.gz': 10234800,
 'part-00001.json.gz': 10230747,
 'part-00002.json.gz': 10227106,
 'part-00003.json.gz': 10233308,
 'part-00004.json.gz': 10233284,
 'part-00005.json.gz': 10235163,
 'part-00006.json.gz': 10232644,
 'part-00007.json.gz': 10236312,
 'part-00008.json.gz': 10228130,
 'part-00009.json.gz': 10229023}

In [12]:
sessionby = ['browser_family', 'os_family', 'device_family']

In [13]:
df = spark.read.json('file://' + filepath)

In [14]:
df.createOrReplaceTempView('df_temp')

In [15]:
etapa2 = {}

In [16]:
for type_session in sessionby:
    df = spark.sql('SELECT ' + type_session + ', COUNT(DISTINCT anonymous_id) qtd FROM df_temp GROUP BY ' + type_session)
    etapa2[type_session] = {r[type_session]: r['qtd'] for r in df.collect()}

In [17]:
etapa2

{'browser_family': {'Edge Mobile': 3274,
  'Other': 2489787,
  'Chrome Mobile': 51526289,
  'PetalBot': 1155,
  'Puffin': 2400,
  'AdsBot-Google': 113642,
  'Firefox Mobile': 125298,
  'Googlebot': 7977489,
  'GooglePlusBot': 139,
  'Yahoo! Slurp': 72,
  'Pinterest': 62,
  'Opera': 174649,
  'Mobile Safari UIWebView': 4365293,
  'Investment Crawler': 3,
  'Chromium': 6110,
  'SMTBot': 284,
  'Mobile Safari': 4707642,
  'IE': 104,
  'Maxthon': 193,
  'UC Browser': 38617,
  'Opera Mobile': 75356,
  'Firefox': 763710,
  'Opera Mini': 223,
  'Android': 404,
  'Facebook': 17789851,
  'Vivaldi': 1257,
  'SeaMonkey': 25,
  'Amazon Silk': 1313,
  'Firefox iOS': 8097,
  'Applebot': 73,
  'Chrome Mobile iOS': 690819,
  'Chrome': 11206459,
  'Edge': 85378,
  'AppleMail': 751,
  'Safari': 161855,
  'Yandex Browser': 1894,
  'Pale Moon (Firefox Variant)': 150,
  'BingPreview': 334,
  'IE Mobile': 68},
 'os_family': {'Other': 7902172,
  'Windows Phone': 3342,
  'Windows XP': 25447,
  'Windows 10': 7

Etapa 2 Completa!

In [18]:
etapa3 = {}

In [19]:
df = spark.read.json('file://' + filepath)

In [21]:
df.write.parquet(filepath + 'sessions3.parquet')

In [24]:
for type_session in sessionby:
    df_session = spark.sql('SELECT '+ type_session + ', anonymous_id, (max(device_sent_timestamp) - min(device_sent_timestamp)) as duration from parquet.`'+ filepath + 'sessions3.parquet` group by '+ type_session + ', anonymous_id')
    df_session.createOrReplaceTempView('df_session_temp')
    df = spark.sql('select '+ type_session + ', format_number(percentile(duration, 0.5), 2) mediana from df_session_temp group by '+ type_session)                   
    etapa3[type_session] = {r[type_session]: r['mediana'] for r in df.collect()}

In [25]:
etapa3

{'browser_family': {'Edge Mobile': '0.00',
  'Other': '0.00',
  'Chrome Mobile': '0.00',
  'PetalBot': '0.00',
  'Puffin': '0.00',
  'AdsBot-Google': '0.00',
  'Firefox Mobile': '0.00',
  'Googlebot': '0.00',
  'GooglePlusBot': '0.00',
  'Yahoo! Slurp': '0.00',
  'Pinterest': '0.00',
  'Opera': '0.00',
  'Mobile Safari UIWebView': '0.00',
  'Investment Crawler': '0.00',
  'Chromium': '0.00',
  'SMTBot': '0.00',
  'Mobile Safari': '0.00',
  'IE': '0.00',
  'Maxthon': '0.00',
  'UC Browser': '0.00',
  'Opera Mobile': '0.00',
  'Firefox': '0.00',
  'Opera Mini': '0.00',
  'Android': '0.00',
  'Facebook': '0.00',
  'Vivaldi': '0.00',
  'SeaMonkey': '0.00',
  'Amazon Silk': '0.00',
  'Firefox iOS': '0.00',
  'Applebot': '0.00',
  'Chrome Mobile iOS': '0.00',
  'Chrome': '0.00',
  'Edge': '0.00',
  'AppleMail': '0.00',
  'Safari': '0.00',
  'Yandex Browser': '0.00',
  'Pale Moon (Firefox Variant)': '0.00',
  'BingPreview': '0.00',
  'IE Mobile': '0.00'},
 'os_family': {'Other': '0.00',
  'Wi

Etapa 3 Finalizada! Aparentemente só temos sessões únicas. 

In [27]:
df = spark.read.json('file:///home/ebbcf/datasets/part-00000.json.gz')

In [28]:
df.createOrReplaceTempView('df_temp')

In [29]:
spark.sql('SELECT browser_family, anonymous_id, (max(device_sent_timestamp) - min(device_sent_timestamp)) as duration from df_temp group by  browser_family, anonymous_id').collect()

[Row(browser_family='Chrome', anonymous_id='c5818cdc-892e-42b5-9fec-c8d73516015c', duration=0),
 Row(browser_family='Mobile Safari', anonymous_id='65dc1fba-50ba-4a49-bbe6-d76015b887fa', duration=0),
 Row(browser_family='Chrome Mobile iOS', anonymous_id='70eaaa71-4a7b-4ecf-b31c-db649fb5e649', duration=0),
 Row(browser_family='Chrome Mobile iOS', anonymous_id='2969a317-5ab0-4f7b-8b05-8dd8465944f5', duration=0),
 Row(browser_family='Chrome Mobile', anonymous_id='f53bf8a6-9a96-4647-b846-07c1a79d9ac7', duration=0),
 Row(browser_family='Facebook', anonymous_id='f9538269-94a4-4c07-9008-0a22a189fe0e', duration=0),
 Row(browser_family='Chrome Mobile', anonymous_id='12bd29cf-7f2b-42b9-8a0c-0f55e84d1253', duration=0),
 Row(browser_family='Chrome', anonymous_id='3d341c0a-cbc5-4e7a-b66a-80dd9fbade0e', duration=0),
 Row(browser_family='Mobile Safari', anonymous_id='4d15c7c4-8639-4b25-a5ed-f2c93062dcd2', duration=0),
 Row(browser_family='Chrome Mobile', anonymous_id='33e472e0-a126-4113-b94c-0a2c79cb1